<a href="https://colab.research.google.com/github/RahulValappile/EmployeeAttrition-ML/blob/main/Solucion_Reto_SC_63_Rahul_Valappile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [7]:
# Subir archivo CSV
uploaded = files.upload()

# Muestra el nombre del archivo subido para confirmar el archivo subido
for filename in uploaded.keys():
    print("Archivo subido:", filename)

Saving bank_marketing_RETO_DS_AS.csv to bank_marketing_RETO_DS_AS (2).csv
Archivo subido: bank_marketing_RETO_DS_AS (2).csv


In [9]:
data = pd.read_csv(filename)

In [11]:
#Exploracion del archivo
print("\nInformación general del DataFrame:")
print(data.info())

print("\nDescripción estadística de variables numéricas:")
print(data.describe())

print("\nCantidad de valores perdidos por variable:")
print(data.isna().sum())


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        9000 non-null   int64 
 1   job        9000 non-null   object
 2   marital    9000 non-null   object
 3   education  9000 non-null   object
 4   default    9000 non-null   object
 5   balance    9000 non-null   int64 
 6   housing    9000 non-null   object
 7   loan       9000 non-null   object
 8   contact    9000 non-null   object
 9   day        9000 non-null   int64 
 10  month      9000 non-null   object
 11  duration   9000 non-null   int64 
 12  campaign   9000 non-null   int64 
 13  pdays      9000 non-null   int64 
 14  previous   9000 non-null   int64 
 15  poutcome   9000 non-null   object
 16  y          9000 non-null   object
dtypes: int64(7), object(10)
memory usage: 1.2+ MB
None

Descripción estadística de variables numéricas:
           

In [13]:
# Preparacion de datos
# Variable objetivo: 'y' (yes/no) -> y_bin (1/0)

data['y_bin'] = data['y'].map({'no': 0, 'yes': 1})

X = data.drop(columns=['y', 'y_bin'])
y = data['y_bin']

# Identificar columnas categóricas y numéricas
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(exclude=['object']).columns.tolist()

print("\nVariables categóricas:", cat_cols)
print("Variables numéricas:", num_cols)


Variables categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
Variables numéricas: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [14]:
# Transformación de variables numéricas sesgadas
# En este tipo de problema típicamente duration, campaign y previous están sesgadas
skew_cols = [col for col in ['duration', 'campaign', 'previous'] if col in num_cols]

for col in skew_cols:
    X[col + '_log'] = np.log1p(X[col])

# Quitamos las columnas originales sesgadas para evitar duplicidad
X = X.drop(columns=skew_cols, errors='ignore')

# Actualizamos listas
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
num_cols = X.select_dtypes(exclude=['object']).columns.tolist()

print("\nVariables numéricas después de la transformación:", num_cols)
print("Variables categóricas después de la transformación:", cat_cols)


Variables numéricas después de la transformación: ['age', 'balance', 'day', 'pdays', 'duration_log', 'campaign_log', 'previous_log']
Variables categóricas después de la transformación: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [15]:
# Particion 60% / 20% / 20%

# 20% para prueba
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# 60% train, 20% val (dentro del 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,  # 25% del 80% total = 20% final
    random_state=42,
    stratify=y_temp
)

print("\nTamaño entrenamiento:", X_train.shape)
print("Tamaño validación:", X_val.shape)
print("Tamaño prueba:", X_test.shape)


Tamaño entrenamiento: (5400, 16)
Tamaño validación: (1800, 16)
Tamaño prueba: (1800, 16)


In [16]:
# Preprocesamiento

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

In [18]:
# Funcion para evaluar modelos

def evaluar_modelo(nombre, modelo, X_val, y_val):
    y_pred = modelo.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)

    print("\n==============================")
    print(f"Modelo: {nombre}")
    print(f"Accuracy validación: {acc:.4f}")
    print("Matriz de confusión:")
    print(cm)
    print("\nReporte clasificación:")
    print(classification_report(y_val, y_pred, target_names=['no', 'yes']))

    return acc

In [19]:
# Modelo 1: Regresion logistica

log_reg = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

log_reg_C01 = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=1000, C=0.1))
])

print("\nEntrenando Regresión Logística...")
log_reg.fit(X_train, y_train)
acc_log = evaluar_modelo("LogReg (C=1.0)", log_reg, X_val, y_val)

print("\nEntrenando Regresión Logística C=0.1...")
log_reg_C01.fit(X_train, y_train)
acc_log_C01 = evaluar_modelo("LogReg (C=0.1)", log_reg_C01, X_val, y_val)


Entrenando Regresión Logística...

Modelo: LogReg (C=1.0)
Accuracy validación: 0.8194
Matriz de confusión:
[[894 148]
 [177 581]]

Reporte clasificación:
              precision    recall  f1-score   support

          no       0.83      0.86      0.85      1042
         yes       0.80      0.77      0.78       758

    accuracy                           0.82      1800
   macro avg       0.82      0.81      0.81      1800
weighted avg       0.82      0.82      0.82      1800


Entrenando Regresión Logística C=0.1...

Modelo: LogReg (C=0.1)
Accuracy validación: 0.8183
Matriz de confusión:
[[887 155]
 [172 586]]

Reporte clasificación:
              precision    recall  f1-score   support

          no       0.84      0.85      0.84      1042
         yes       0.79      0.77      0.78       758

    accuracy                           0.82      1800
   macro avg       0.81      0.81      0.81      1800
weighted avg       0.82      0.82      0.82      1800



In [20]:
# Modelo 2: Red neuronal

mlp = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', MLPClassifier(
        hidden_layer_sizes=(32, 16),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        max_iter=300,
        random_state=42
    ))
])

mlp_simple = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', MLPClassifier(
        hidden_layer_sizes=(16,),
        activation='relu',
        solver='adam',
        alpha=0.001,
        max_iter=300,
        random_state=42
    ))
])

print("\nEntrenando Red Neuronal (32,16)...")
mlp.fit(X_train, y_train)
acc_mlp = evaluar_modelo("MLP (32,16)", mlp, X_val, y_val)

print("\nEntrenando Red Neuronal simple (16)...")
mlp_simple.fit(X_train, y_train)
acc_mlp_simple = evaluar_modelo("MLP (16)", mlp_simple, X_val, y_val)


Entrenando Red Neuronal (32,16)...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(



Modelo: MLP (32,16)
Accuracy validación: 0.8089
Matriz de confusión:
[[847 195]
 [149 609]]

Reporte clasificación:
              precision    recall  f1-score   support

          no       0.85      0.81      0.83      1042
         yes       0.76      0.80      0.78       758

    accuracy                           0.81      1800
   macro avg       0.80      0.81      0.81      1800
weighted avg       0.81      0.81      0.81      1800


Entrenando Red Neuronal simple (16)...

Modelo: MLP (16)
Accuracy validación: 0.8456
Matriz de confusión:
[[864 178]
 [100 658]]

Reporte clasificación:
              precision    recall  f1-score   support

          no       0.90      0.83      0.86      1042
         yes       0.79      0.87      0.83       758

    accuracy                           0.85      1800
   macro avg       0.84      0.85      0.84      1800
weighted avg       0.85      0.85      0.85      1800



/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
# Seleccion del mejor modelo

results = {
    "LogReg (C=1.0)": (acc_log, log_reg),
    "LogReg (C=0.1)": (acc_log_C01, log_reg_C01),
    "MLP (32,16)": (acc_mlp, mlp),
    "MLP (16)": (acc_mlp_simple, mlp_simple)
}

best_name = max(results, key=lambda k: results[k][0])
best_acc, best_model = results[best_name]

print("Mejor modelo en validacion:")
print(f"Modelo: {best_name}")
print(f"Accuracy: {best_acc:.4f}")

MEJOR MODELO EN VALIDACIÓN:
Modelo: MLP (16)
Accuracy: 0.8456


In [24]:
# Evaluacion final en prueba

y_test_pred = best_model.predict(X_test)

acc_test = accuracy_score(y_test, y_test_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

print("=== Desempeno final en prueba===")
print(f"Accuracy en prueba: {acc_test:.4f}")
print("\nMatriz de confusión (prueba):")
print(cm_test)
print("\nReporte clasificación (prueba):")
print(classification_report(y_test, y_test_pred, target_names=['no', 'yes']))


=== Desempeno final en prueba===
Accuracy en prueba: 0.8494

Matriz de confusión (prueba):
[[874 169]
 [102 655]]

Reporte clasificación (prueba):
              precision    recall  f1-score   support

          no       0.90      0.84      0.87      1043
         yes       0.79      0.87      0.83       757

    accuracy                           0.85      1800
   macro avg       0.85      0.85      0.85      1800
weighted avg       0.85      0.85      0.85      1800

